In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec

import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import KFold

from sklearn.base import clone as sklearn_clone

from datasets import Dataset, DatasetDict
import pandas as pd
import torch

from transformers import AutoTokenizer

from adapters import AutoAdapterModel
from transformers import AutoTokenizer

from adapters import BertAdapterModel

from transformers import  BertConfig

from adapters import SeqBnConfig #PfeifferConfig

from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error

torch.cuda.is_available()

True

In [2]:
# !conda install -c conda-forge huggingface_hub
# !pip install --force-reinstall charset-normalizer==3.1.0
# !pip install accelerate -U

## Lodad data

In [3]:
with open('../datasets/used_data/03_bert_like_models/02_topics.npy', 'rb') as f:
    df_topics = pd.DataFrame(np.load(f), columns=['topic'])
df_topics.shape

(6529, 1)

In [4]:
df = pd.read_parquet('../datasets/used_data/03_bert_like_models/01_basic_info_data.parquet')

df['sentence'] = df['text'].apply(lambda x: x.replace('\n',' ').strip())
df['labels'] = df['assestment'].astype(int)

df = df.sample(frac=1, random_state=111)

df = df[df.index.isin(df_topics.index)]
df.shape

(6529, 5)

In [5]:
tok = AutoTokenizer.from_pretrained('allegro/herbert-large-cased')

In [6]:
labels = sorted(set(df['labels']))
num_labels = len(labels)
label2id = dict(zip(labels,range(num_labels)))
id2label = {v: k for k, v in label2id.items()}

def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    batch['labels'] = np.array(batch['labels'])
    return tok(text=batch['sentence'], max_length=64, truncation=True, padding="max_length")

### Kfold

In [7]:
cv_fold = []
cv_fold_i = []

for i in df_topics['topic'].unique().reshape(10,-1):
    train_cv = df_topics.index[ ~np.isin(df_topics["topic"], i) ].values
    test_cv = df_topics.index[ np.isin(df_topics["topic"], i) ].values
    
    cv_fold.append( [train_cv, test_cv])

In [8]:
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(df_topics)

cv_Kfold = []
cv_Kfold_i = []

for train_index, test_index in kf.split(df_topics):
    train_cv = df_topics.iloc[ train_index, : ].index.values
    test_cv = df_topics.iloc[ test_index, : ].index.values
    
    cv_Kfold.append( [train_cv, test_cv])

## Experiments

In [9]:
config = BertConfig.from_pretrained(
    'allegro/herbert-large-cased',
    num_labels=num_labels
)

adapter_config = SeqBnConfig() #PfeifferConfig()


training_args = TrainingArguments(
    remove_unused_columns=False, # ensure the dataset labels are properly passed to the model
    
    fp16=True,
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    label_smoothing_factor=0.0,
    weight_decay=0.05,
    warmup_steps=600,
    output_dir="./training_output",
    overwrite_output_dir=True,
    
#     logging_strategy="no"
    
#     logging_steps=200,
#     logging_first_step=False,
#     logging_dir='./tb_logs',
#     evaluation_strategy='steps',
#     report_to='tensorboard',
#     save_steps=200
)

def compute_accuracy_f1score(p: EvalPrediction):
    preds = [id2label[pred_id] for pred_id in np.argmax(p.predictions, axis=1)]
    target_labels = [id2label[label_id] for label_id in p.label_ids]
    return {"accuracy": accuracy_score(target_labels,preds),
           "f1score": f1_score(target_labels,preds)}

In [10]:
cv = cv_fold

In [11]:
acc = []
f1 = []

j = 1

for train_cv, test_cv in cv:
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(df[df.index.isin(train_cv)])
    ds['valid'] = Dataset.from_pandas(df[df.index.isin(test_cv)])
    
    
    ds = ds.map(encode_batch, batched=True, batch_size=len(ds['train']))
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    
    
    model = BertAdapterModel.from_pretrained('allegro/herbert-large-cased', config=config).to('cuda')
    
    adapter_name='fake_news_'+str(j)
    
    model.add_adapter(adapter_name,config=adapter_config)
    model.add_classification_head(
        adapter_name,
        id2label=id2label,
        num_labels=num_labels
      )

    model.train_adapter(adapter_name)
    
    
    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        compute_metrics=compute_accuracy_f1score,
    )

    trainer.train()
    
    eval_out = trainer.evaluate()
    
    acc.append(eval_out['eval_accuracy'])
    f1.append(eval_out['eval_f1score'])
    
    j+=1

Map:   0%|          | 0/5906 [00:00<?, ? examples/s]

Map:   0%|          | 0/623 [00:00<?, ? examples/s]

Step,Training Loss
500,0.579900
1000,0.535000
1500,0.513600
2000,0.505300
2500,0.479000
3000,0.475900
3500,0.449700
4000,0.437500
4500,0.418900
5000,0.389200


Map:   0%|          | 0/5877 [00:00<?, ? examples/s]

Map:   0%|          | 0/652 [00:00<?, ? examples/s]

...

Module weights saved in ./training_output/checkpoint-4000/fake_news_9/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-4000/fake_news_10/adapter_config.json
Module weights saved in ./training_output/checkpoint-4000/fake_news_10/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-4000/fake_news_10/head_config.json
Module weights saved in ./training_output/checkpoint-4000/fake_news_10/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-4000/fake_news_1/head_config.json
Module weights saved in ./training_output/checkpoint-4000/fake_news_1/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-4000/fake_news_2/head_config.json
Module weights saved in ./training_output/checkpoint-4000/fake_news_2/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-4000/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-4000/fake_news_3/pytorch_model_head.bin
Configur

Configuration saved in ./training_output/checkpoint-5000/fake_news_2/head_config.json
Module weights saved in ./training_output/checkpoint-5000/fake_news_2/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5000/fake_news_3/adapter_config.json
Module weights saved in ./training_output/checkpoint-5000/fake_news_3/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-5000/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-5000/fake_news_3/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5000/fake_news_4/adapter_config.json
Module weights saved in ./training_output/checkpoint-5000/fake_news_4/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-5000/fake_news_4/head_config.json
Module weights saved in ./training_output/checkpoint-5000/fake_news_4/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5000/fake_news_5/adapter_config.json
Module weights s

Module weights saved in ./training_output/checkpoint-5500/fake_news_10/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_10/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_10/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_1/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_1/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_2/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_2/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_3/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_4/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_4/pytorch_model_head.bin
Configuratio

Configuration saved in ./training_output/checkpoint-6500/fake_news_3/adapter_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_3/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_3/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_4/adapter_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_4/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_4/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_4/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_5/adapter_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_5/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_5/head_config.json
Module weights save

Module weights saved in ./training_output/checkpoint-7000/fake_news_10/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_1/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_1/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_2/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_2/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_3/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_4/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_4/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_5/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_5/pytorch_model_head.bin
Configurati

In [12]:
acc = np.array(acc)
f1 = np.array(f1)

print(
    'adapters',
    f'Accuracy {acc.mean():.3f}+-{acc.std():.3f}',
    f'F1 Score {f1.mean():.3f}+-{f1.std():.3f}',
    f' {acc.mean():.3f}+-{acc.std():.3f} | {f1.mean():.3f}+-{f1.std():.3f}'
)

adapters Accuracy 0.761+-0.029 F1 Score 0.754+-0.063  0.761+-0.029 | 0.754+-0.063


In [13]:
cv = cv_Kfold

In [14]:
acc = []
f1 = []

for train_cv, test_cv in cv:
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(df[df.index.isin(train_cv)])
    ds['valid'] = Dataset.from_pandas(df[df.index.isin(test_cv)])
    
    
    ds = ds.map(encode_batch, batched=True, batch_size=len(ds['train']))
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    
    
    model = BertAdapterModel.from_pretrained('allegro/herbert-large-cased', config=config).to('cuda')

    adapter_name='fake_news_'+str(j)
    
    model.add_adapter(adapter_name,config=adapter_config)
    model.add_classification_head(
        adapter_name,
        id2label=id2label,
        num_labels=num_labels
      )

    model.train_adapter(adapter_name)
    
    
    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["valid"],
        compute_metrics=compute_accuracy_f1score,
    )

    trainer.train()
    
    eval_out = trainer.evaluate()
    
    acc.append(eval_out['eval_accuracy'])
    f1.append(eval_out['eval_f1score'])
    
    j += 1

Map:   0%|          | 0/5876 [00:00<?, ? examples/s]

Map:   0%|          | 0/653 [00:00<?, ? examples/s]

Some weights of BertAdapterModel were not initialized from the model checkpoint at allegro/herbert-large-cased and are newly initialized: ['heads.fake_news_7.1.bias', 'heads.fake_news_5.1.weight', 'heads.fake_news_9.4.bias', 'heads.fake_news_10.4.weight', 'heads.fake_news_1.1.weight', 'heads.fake_news_1.1.bias', 'heads.fake_news_8.1.bias', 'heads.fake_news_4.4.bias', 'heads.fake_news_2.1.bias', 'heads.fake_news_8.4.bias', 'heads.fake_news_4.1.weight', 'heads.fake_news_7.4.weight', 'heads.fake_news_9.4.weight', 'heads.fake_news_8.1.weight', 'heads.fake_news_10.4.bias', 'heads.fake_news_7.4.bias', 'heads.fake_news_2.1.weight', 'heads.fake_news_7.1.weight', 'heads.fake_news_9.1.bias', 'heads.fake_news_4.1.bias', 'heads.fake_news_10.1.weight', 'heads.fake_news_3.1.bias', 'heads.fake_news_6.1.weight', 'heads.fake_news_3.4.weight', 'heads.fake_news_5.4.bias', 'heads.fake_news_4.4.weight', 'heads.fake_news_8.4.weight', 'heads.fake_news_5.4.weight', 'heads.fake_news_5.1.bias', 'heads.fake_news

Step,Training Loss
500,0.576800
1000,0.528300
1500,0.517500
2000,0.497300
2500,0.502500
3000,0.469500
3500,0.444400
4000,0.429500
4500,0.415600
5000,0.370500


Map:   0%|          | 0/5876 [00:00<?, ? examples/s]

...

Module weights saved in ./training_output/checkpoint-5500/fake_news_20/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_20/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_20/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_1/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_1/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_2/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_2/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_3/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-5500/fake_news_4/head_config.json
Module weights saved in ./training_output/checkpoint-5500/fake_news_4/pytorch_model_head.bin
Configuratio

Configuration saved in ./training_output/checkpoint-6000/fake_news_13/adapter_config.json
Module weights saved in ./training_output/checkpoint-6000/fake_news_13/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6000/fake_news_13/head_config.json
Module weights saved in ./training_output/checkpoint-6000/fake_news_13/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6000/fake_news_14/adapter_config.json
Module weights saved in ./training_output/checkpoint-6000/fake_news_14/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6000/fake_news_14/head_config.json
Module weights saved in ./training_output/checkpoint-6000/fake_news_14/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6000/fake_news_15/adapter_config.json
Module weights saved in ./training_output/checkpoint-6000/fake_news_15/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6000/fake_news_15/head_config.json
Module w

Module weights saved in ./training_output/checkpoint-6500/fake_news_5/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_6/adapter_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_6/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_6/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_6/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_7/adapter_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_7/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_7/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_7/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_8/adapter_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_8/pytorch_adapter.bin
Configuratio

Configuration saved in ./training_output/checkpoint-6500/fake_news_16/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_16/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_17/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_17/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_18/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_18/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_19/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_19/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-6500/fake_news_20/head_config.json
Module weights saved in ./training_output/checkpoint-6500/fake_news_20/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-7000
Configuration saved in ./training_

Module weights saved in ./training_output/checkpoint-7000/fake_news_1/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_2/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_2/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_3/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_3/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_4/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_4/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_5/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_5/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-7000/fake_news_6/head_config.json
Module weights saved in ./training_output/checkpoint-7000/fake_news_6/pytorch_model_head.bin
Configuratio

In [15]:
acc = np.array(acc)
f1 = np.array(f1)

print(
    'adapters',
    f'Accuracy {acc.mean():.3f}+-{acc.std():.3f}',
    f'F1 Score {f1.mean():.3f}+-{f1.std():.3f}',
    f' {acc.mean():.3f}+-{acc.std():.3f} | {f1.mean():.3f}+-{f1.std():.3f}'
)

adapters Accuracy 0.765+-0.015 F1 Score 0.766+-0.018  0.765+-0.015 | 0.766+-0.018
